In [ ]:
%load_ext autoreload
%autoreload 2


In [ ]:
from pathlib import Path
from piano_vision_fingering_generator.midi.generate_fingerings import (
    GeneratedPianoFingerings,
    PianoFingering,
)
from piano_vision_fingering_generator.piano_vision.models import PianoVisionSong
from piano_vision_fingering_generator import generate_piano_fingerings, HandSize
from piano_vision_fingering_generator.piano_vision.io import read_piano_vision_json

midi_song_path = Path("./data/zeldas_lullaby/zeldas_lullaby.mid")
piano_vision_song_path = Path("./data/zeldas_lullaby/zeldas_lullaby_piano_vision.json")
song: PianoVisionSong = read_piano_vision_json(piano_vision_song_path)

fingerings: GeneratedPianoFingerings = generate_piano_fingerings(
    midi_song_path, HandSize.MEDIUM
)


In [ ]:
from piano_vision_fingering_generator.main import (
    generate_fingerings_and_update_piano_vision_song,
)


generate_fingerings_and_update_piano_vision_song(
    piano_vision_song_path, midi_song_path, HandSize.MEDIUM
)


In [ ]:
for fingering in fingerings.left:
    if not fingering.measure:
        continue
    measure = song.tracks_v2.left[fingering.measure]
    for note in measure.notes


In [ ]:
from piano_vision_fingering_generator.main import (
    generate_fingerings_and_update_piano_vision_song,
)
from piano_vision_fingering_generator import set_logging_level

set_logging_level("DEBUG")
from tqdm.contrib.logging import logging_redirect_tqdm

with logging_redirect_tqdm():
    song = generate_fingerings_and_update_piano_vision_song(
        piano_vision_song_path, midi_song_path, HandSize.MEDIUM
    )


# Music21 Midi Exploration


In [ ]:
from piano_vision_fingering_generator.constants import Finger, NoteLengthType
from piano_vision_fingering_generator.piano_vision.models import Note
import music21 as m21


In [39]:
from typing import cast

from music21.midi import MidiEvent, MidiTrack

from piano_vision_fingering_generator.constants import (
    Finger,
    NoteLengthType,
)
from piano_vision_fingering_generator.midi.generate_fingerings import (
    is_chord,
    is_note,
    is_rest,
)
from piano_vision_fingering_generator.piano_vision.models import (
    PianoVisionMeasure,
    PianoVisionSong,
    Direction,
    Rest,
    Note,
    SupportingTrack,
    SupportingTrackMidi,
    TracksV2,
)
from pathlib import Path
from music21 import converter, environment
from pianoplayer.scorereader import reader
from pianoplayer.hand import Hand
import music21 as m21

environment.UserSettings()


midi_song_path = Path("./data/zeldas_lullaby/zeldas_lullaby.mid")
midi_song = converter.parse(midi_song_path)
mf = m21.midi.MidiFile()
mf.open(str(midi_song_path))
midi_song_ticks_per_quarter_note = mf.ticksPerQuarterNote
mf.close()


def pv_note_from_m21_note(
    m21_note: m21.note.Note,
    measure: m21.stream.Measure,
    metrenome: m21.tempo.MetronomeMark,
    measure_note_count: int,
    note_id: int,
    note_hand: str,
    fingering: Finger,
) -> Note:

    octave: int = int(m21_note.pitch.octave)  # type: ignore
    # if fingering == Finger.NOT_SET:
    # raise ValueError(f"Fingering not set for note {m21_note}")
    note = Note(
        id=f"{note_hand}{note_id}",
        note=m21_note.pitch.midi,
        duration=metrenome.durationToSeconds(m21_note.duration),
        durationTicks=0,
        finger=fingering,
        group=-1,
        start=metrenome.durationToSeconds(m21_note.offset),
        end=metrenome.durationToSeconds(
            m21_note.offset + m21_note.duration.quarterLength
        ),
        measureBars=0,
        noteOffVelocity=0,
        ticksStart=0,
        velocity=0,
        noteName=m21_note.pitch.name,
        octave=octave,
        notePitch=m21_note.pitch.name,
        noteLengthType=NoteLengthType(m21_note.duration.type),
        measureInd=measure.number,
        noteMeasureInd=measure_note_count,
    )
    return note


def build_supporting_tracks(midi_song_path: Path) -> SupportingTrack:
    mf = m21.midi.MidiFile()
    mf.open(str(midi_song_path))
    mf.read()
    print(mf.ticksPerQuarterNote)
    for track in mf.tracks:
        current_time = 0
        track: MidiTrack = cast(m21.midi.MidiTrack, track)
        for event in track.events:
            if event.type == m21.midi.MetaEvents.TIME_SIGNATURE:
                print(event.data)
            if event.isNoteOn():
                note_pitch = event.pitch
                note_velocity = event.velocity
                # print(note_pitch, note_velocity, current_time)
            if event.isDeltaTime():
                current_time += event.time
            event: MidiEvent = cast(m21.midi.MidiEvent, event)

    mf.close()
    supporting_track = SupportingTrack(myInstrument=-5, theirInstrument=0, notes=[])
    return supporting_track


build_supporting_tracks(midi_song_path)


def build_pv_measures(midi_song_path: Path) -> TracksV2:
    midi_song = converter.parse(midi_song_path)
    # mf = m21.midi.MidiFile()
    # mf.open(str(midi_song_path))
    # midi_song_ticks_per_quarter_note = mf.ticksPerQuarterNote
    # mf.close()

    rhand = Hand("right", "M")
    rhand.noteseq = reader(midi_song, beam=0)
    rhand.verbose = False
    rhand.generate()
    lhand = Hand("left", "M")
    lhand.verbose = False
    lhand.noteseq = reader(midi_song, beam=1)
    lhand.generate()

    right_pv_measures = []
    left_pv_measures = []
    time_sig = None
    is_first_loop = True
    hands = ("right", "left")
    note_count = 0
    for part in midi_song.parts:  # type: ignore
        hand = "right" if is_first_loop else "left"
        is_first_loop = False
        for measure_data in part.secondsMap:
            measure_note_count = 0
            min_note_id = 200
            max_note_id = 0
            measure: m21.stream.Measure = measure_data["element"]  # type: ignore
            # print(measure.timeSignature)
            if measure.timeSignature and time_sig != measure.timeSignature:
                time_sig = measure.timeSignature
            # if not time_sig:
            # raise ValueError("No time signature found")
            metrenome_and_boundaries: list[
                tuple[float, float, m21.tempo.MetronomeMark]
            ] = measure.metronomeMarkBoundaries()
            notes: list[Note] = []
            rests: list[Rest] = []
            metrenome = None
            for el in measure.notesAndRests:
                for lower_bound, upper_bound, _metrenome in metrenome_and_boundaries:
                    metrenome_duration = 0
                    if lower_bound <= el.offset <= upper_bound:
                        metrenome = _metrenome
                        metrenome_duration = metrenome.durationToSeconds(el.duration)
                        metrenome_duration_ticks = 0
                if not metrenome:
                    raise ValueError("No metrenome found")
                if is_note(el):
                    note_count += 1
                    measure_note_count += 1
                    fingering = Finger.NOT_SET
                    for art in el.articulations:
                        if isinstance(art, m21.articulations.Fingering):
                            fingering = Finger(art.fingerNumber)
                        else:
                            print(art)
                    if el.pitch.midi < min_note_id:
                        min_note_id = el.pitch.midi
                    if el.pitch.midi > max_note_id:
                        max_note_id = el.pitch.midi
                    note: Note = pv_note_from_m21_note(
                        el,
                        measure,
                        metrenome,
                        measure_note_count,
                        note_count,
                        "l",
                        fingering,
                    )
                    notes.append(note)
                elif is_rest(el):
                    rest = Rest(
                        noteLengthType=NoteLengthType(el.duration.type),
                        time=metrenome.durationToSeconds(el.offset),
                    )
                    rests.append(rest)
                elif is_chord(el):
                    fingerings = []
                    for art in el.articulations:
                        if isinstance(art, m21.articulations.Fingering):
                            fingerings.append(Finger(art.fingerNumber))
                        else:
                            print(art)
                    rests.append(rest)
                    for chord_note, fingering in zip(el.notes, fingerings):
                        measure_note_count += 1
                        note_count += 1
                        note = pv_note_from_m21_note(
                            chord_note,
                            measure,
                            metrenome,
                            measure_note_count,
                            note_count,
                            "l",
                            fingering,
                        )
                        notes.append(note)
            pv_measure = PianoVisionMeasure(
                direction=Direction.UP,
                time=measure_data["offsetSeconds"],
                timeEnd=measure_data["endTimeSeconds"],
                max=max_note_id,
                min=min_note_id,
                measureTicksEnd=0,
                measureTicksStart=0,
                notes=notes,
                rests=rests,
                timeSignature=[time_sig.numerator, time_sig.denominator],
            )
            if hand == "right":

                right_pv_measures.append(pv_measure)
            else:
                left_pv_measures.append(pv_measure)
    return TracksV2(right=right_pv_measures, left=left_pv_measures)


tracks_v2 = build_pv_measures(midi_song_path)
# for pv_measure in pv_measures:
#     print(pv_measure)

# PianoVisionSong(
#     tracks_v2=TracksV2(
#         right=pv_measures,
#         left=pv_measures,
#     ),
#     technicalGroups=[],
#     positionGroups=[],
# )


480
b'\x03\x02\x18\x08'
Your hand span set to size-M which is 17.22 cm
(max relaxed distance between thumb and pinkie)
Reading beam 0 with 73 objects in stream.
Your hand span set to size-M which is 17.22 cm
(max relaxed distance between thumb and pinkie)
Reading beam 1 with 102 objects in stream.


In [ ]:
[part.show() for part in midi_song.parts]


In [ ]:
midi_song_path = Path("./data/zeldas_lullaby/zeldas_lullaby.mid")
midi_song2 = converter.parse(midi_song_path)
rhand = Hand("right", "M")
rhand.noteseq = reader(midi_song2, beam=0)
rhand.verbose = False
rhand.generate()
lhand = Hand("left", "M")
lhand.verbose = False
lhand.noteseq = reader(midi_song2, beam=1)
lhand.generate()


In [ ]:
[part.show() for part in midi_song2.parts]
